In [10]:
import jsonschema
import jsonpointer
import json
import awswrangler as wr
import boto3

In [3]:
boto3.setup_default_session(profile_name='pawprint')

In [4]:
df = wr.s3.read_json('s3://pawprint-raw/genome/dataset_reports/date=2023-11-28/', lines=True)

In [5]:
df.head()

,reports,total_count
0,"[{'accession': 'GCA_014108235.1', 'current_acc...",5
0,"[{'accession': 'GCA_903992545.1', 'current_acc...",10
0,"[{'accession': 'GCA_030015415.1', 'current_acc...",12
0,"[{'accession': 'GCA_027574615.1', 'current_acc...",14
0,"[{'accession': 'GCA_003508835.1', 'current_acc...",2


In [6]:
final_list = []
for i in df['reports']:
    final_list.extend(i)

In [35]:
def fill_missing_fields(data, schema):
    for field_name, field_schema in schema.get('properties', {}).items():
        print('processing field', field_name)
        if field_schema.get("type") == "object":
            print('processing object', field_schema)
            # Recursively handle nested fields
            try:
                nested_data = jsonpointer.resolve_pointer(data, f"/{field_name}")
            except:
                nested_data = {}  # Create the nested object if missing
            nested_data = fill_missing_fields(nested_data, field_schema)
            jsonpointer.set_pointer(data, f"/{field_name}", nested_data)
        elif field_schema.get("type") == "array":
            # Recursively handle nested fields
            try:
                nested_data = jsonpointer.resolve_pointer(data, f"/{field_name}")
            except:
                nested_data = []  # Create the nested object if missing
            for i in range(len(nested_data)):
                nested_data[i] = fill_missing_fields(nested_data[i], field_schema.get("items"))
            jsonpointer.set_pointer(data, f"/{field_name}", nested_data)
        else:
            if field_name not in data:
                jsonpointer.set_pointer(data, f"/{field_name}", None)

    return data

In [27]:
record = final_list[0]

In [42]:
with open('../config/schemas/genome_schema.json') as f:
    schema = json.load(f)

In [43]:
record_new = fill_missing_fields(record, schema)

processing field accession
processing field current_accession
processing field paired_accession
processing field source_database
processing field organism
processing object {'type': 'object', 'properties': {'tax_id': {'type': 'integer'}, 'sci_name': {'type': 'string'}, 'organism_name': {'type': 'string'}, 'common_name': {'type': 'string'}, 'lineage': {'type': 'array', 'items': {'type': 'object', 'properties': {'tax_id': {'type': 'integer'}, 'name': {'type': 'string'}}}}, 'strain': {'type': 'string'}, 'pangolin_classification': {'type': 'string'}, 'infraspecific_names': {'type': 'object', 'properties': {'breed': {'type': 'string'}, 'cultivar': {'type': 'string'}, 'ecotype': {'type': 'string'}, 'isolate': {'type': 'string'}, 'sex': {'type': 'string'}, 'strain': {'type': 'string'}}}}}
processing field tax_id
processing field sci_name
processing field organism_name
processing field common_name
processing field lineage
processing field strain
processing field pangolin_classification
process

In [44]:
record_new

{'accession': 'GCA_014108235.1',
 'current_accession': 'GCA_014108235.1',
 'paired_accession': 'GCF_014108235.1',
 'source_database': 'SOURCE_DATABASE_GENBANK',
 'organism': {'tax_id': 51298,
  'organism_name': 'Myotis myotis',
  'infraspecific_names': {'isolate': 'mMyoMyo1',
   'sex': 'female',
   'breed': None,
   'cultivar': None,
   'ecotype': None,
   'strain': None},
  'sci_name': None,
  'common_name': None,
  'lineage': [],
  'strain': None,
  'pangolin_classification': None},
 'assembly_info': {'assembly_level': 'Scaffold',
  'assembly_status': 'current',
  'paired_assembly': {'accession': 'GCF_014108235.1',
   'status': 'current',
   'annotation_name': 'NCBI Myotis myotis Annotation Release 100',
   'changed': 'chromosome MT.',
   'only_genbank': None,
   'only_refseq': None,
   'manual_diff': None},
  'assembly_name': 'mMyoMyo1.p',
  'assembly_type': 'haploid',
  'bioproject_lineage': [{'bioprojects': [{'accession': 'PRJNA628559',
      'title': 'Myotis myotis isolate:mMyoMy